# Table 4. Factors associated with VAS reduction (Medical cases, all ages)

In [ ]:
import pandas as pd
import numpy as np
import re
import statsmodels.api as sm
import warnings
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/analgesia/data/rega_data/trauma_categories_Rega Pain Study15.09.2025_v2.xlsx'
medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/analgesia/data/rega_data/rega_physician_list_09102025.xlsx'
meta_medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/analgesia/data/medreg_extraction/joined_final_complete_extractions_20251008_221735.xlsx'
restrict_to_primary = True
restrict_to_medical = True

In [ ]:
data_df = pd.read_excel(data_path)
medic_df = pd.read_excel(medic_data_path)
meta_medic_df = pd.read_excel(meta_medic_data_path)

medic_df['full_name'] = medic_df['Mitglieder mit Einsatzfunktion'].str.replace(' (Flugarzt/Flugärztin)', '')
medic_df.drop_duplicates(subset=['Mitglieder mit Einsatzfunktion'], inplace=True)
medic_df = medic_df.merge(meta_medic_df, how='left', on='full_name')
medic_df.rename(columns={'Sex m/w': 'physician_sex'}, inplace=True)
data_df = data_df.merge(medic_df, how='left', left_on='Mitglieder mit Einsatzfunktion', right_on='Mitglieder mit Einsatzfunktion')

data_df = data_df.drop_duplicates(subset=['SNZ Ereignis Nr. '])
data_df = data_df[data_df['VAS_on_scene'] > 3]

n_missing_arrival = data_df['VAS_on_arrival'].isna().sum()
print(f'Excluded {n_missing_arrival} patients with missing VAS_on_arrival')
data_df = data_df.dropna(subset=['VAS_on_arrival'])

if restrict_to_medical:
    n_non_medical = data_df[data_df['Einteilung (reduziert)'] != 'Krankheit'].shape[0]
    print(f'Excluded {n_non_medical} non-medical patients')
    data_df = data_df[data_df['Einteilung (reduziert)'] == 'Krankheit']

if restrict_to_primary:
    n_secondary = data_df[data_df['Einsatzart'] != 'Primär'].shape[0]
    print(f'Excluded {n_secondary} secondary transport patients')
    data_df = data_df[data_df['Einsatzart'] == 'Primär']

if 'NACA' not in data_df.columns and 'NACA (nummerisch)' in data_df.columns:
    data_df['NACA'] = data_df['NACA (nummerisch)']

In [ ]:
def univariate_linear_regression(df, outcome_var, predictor_vars):
    results = []
    for var in predictor_vars:
        X = df[[var]].copy()
        y = df[outcome_var]
        X_with_const = sm.add_constant(X)
        try:
            model = sm.OLS(y, X_with_const).fit()
            coef = model.params[var]
            se = model.bse[var]
            ci_lower = model.conf_int().loc[var, 0]
            ci_upper = model.conf_int().loc[var, 1]
            p_value = model.pvalues[var]
            r_squared = model.rsquared
            results.append({
                'Variable': var,
                'Coefficient': coef,
                'SE': se,
                'CI_lower': ci_lower,
                'CI_upper': ci_upper,
                'P_value': p_value,
                'R_squared': r_squared,
                'Coef_CI': f"{coef:.3f} ({ci_lower:.3f} to {ci_upper:.3f})",
                'P_formatted': f"{p_value:.3f}" if p_value >= 0.001 else "<0.001"
            })
        except Exception as e:
            print(f"Error with variable {var}: {e}")
    return pd.DataFrame(results)

def multivariate_linear_regression(df, outcome_var, predictor_vars, normalize=False):
    X = df[predictor_vars].copy()
    y = df[outcome_var]
    if normalize:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        X = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
    X_with_const = sm.add_constant(X)
    model = sm.OLS(y, X_with_const).fit()
    results = []
    for var in X.columns:
        coef = model.params[var]
        se = model.bse[var]
        ci_lower = model.conf_int().loc[var, 0]
        ci_upper = model.conf_int().loc[var, 1]
        p_value = model.pvalues[var]
        results.append({
            'Variable': var,
            'Coefficient': coef,
            'SE': se,
            'CI_lower': ci_lower,
            'CI_upper': ci_upper,
            'P_value': p_value,
            'Coef_CI': f"{coef:.3f} ({ci_lower:.3f} to {ci_upper:.3f})",
            'P_formatted': f"{p_value:.3f}" if p_value >= 0.001 else "<0.001"
        })
    return pd.DataFrame(results), model

def drop_zero_variance(df, vars_list):
    kept = [v for v in vars_list if df[v].nunique(dropna=True) > 1]
    dropped = [v for v in vars_list if v not in kept]
    if dropped:
        print(f'Dropped zero-variance variables: {dropped}')
    return kept

In [ ]:
df = data_df.copy()
df['vas_reduction'] = df['VAS_on_scene'] - df['VAS_on_arrival']

df['age'] = df['Alter ']
df['male_patient'] = (df['Geschlecht'] == 'Männlich').astype(int)
df['male_physician'] = (df['physician_sex'] == 'm').astype(int)

df['event_year'] = pd.to_datetime(df['Ereignisdatum'], format='%d.%m.%Y').dt.year
df['physician_age'] = df['event_year'] - df['year_of_birth']
df['physician_licence_year'] = df['licence_date'].apply(lambda x: str(x).split('.')[-1] if '.' in str(x) else str(x))
df['physician_experience_years'] = df['event_year'] - pd.to_numeric(df['physician_licence_year'], errors='coerce')

df['physician_anesthesiologist'] = df['specialist_qualifications'].str.contains('Anaesthesiology', na=False).astype(int)
df['physician_intensivist'] = df['specialist_qualifications'].str.contains('Intensive care medicine', na=False).astype(int)
df['physician_internist'] = df['specialist_qualifications'].str.contains('General Internal Medicine|General medical practitioner', na=False).astype(int)

df['primary_mission'] = (df['Einsatzart'] == 'Primär').astype(int)
df['night_mission'] = (df['Tag oder Nacht'] == 'Nacht').astype(int)
df['winter_season'] = np.where(df['Monat'].isin(['Oktober', 'November', 'Dezember', 'Januar', 'Februar', 'März']), 1, 0).astype(int)
df['winch_extraction'] = df['Bergungen'].str.contains('Winde', na=False).astype(int)
df['vas_scene'] = df['VAS_on_scene']
df['mission_duration'] = (pd.to_datetime(df['Übergabezeit'], format='%d.%m.%Y %H:%M:%S') - pd.to_datetime(df['Erstbefund'], format='%d.%m.%Y %H:%M:%S')).dt.total_seconds() / 60

df['fentanyl_dose'] = 0
df['ketamine_dose'] = 0
df['esketamine_dose'] = 0
df['morphine_dose'] = 0
df['Alle Medikamente'] = df['Alle Medikamente'].str.replace(',', ';')
for i, row in df.iterrows():
    if pd.isna(row['Alle Medikamente']) or row['Alle Medikamente'] == 0:
        continue
    for analgetic in row['Alle Medikamente'].split(';'):
        if analgetic.strip() == '':
            continue
        if '7IE' in analgetic:
            continue
        analgetic = analgetic.replace('mcg', '').replace('mg', '').strip()
        if 'Fentanyl' in analgetic and '/h' not in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip()
            df.at[i, 'fentanyl_dose'] += float(dose)
        elif 'Fentanyl' in analgetic and '/h' in analgetic:
            dose = analgetic.split('Fentanyl')[-1].strip().replace('/h', '')
            dose = float(dose) * df.at[i, 'mission_duration']
            df.at[i, 'fentanyl_dose'] += float(dose)
        elif 'Ketamin' in analgetic or 'Ketamine' in analgetic:
            dose = analgetic.split('Ketamin')[-1].strip()
            df.at[i, 'ketamine_dose'] += float(dose)
        elif 'Esketamin' in analgetic:
            dose = analgetic.split('Esketamin')[-1].strip()
            df.at[i, 'esketamine_dose'] += float(dose)
        elif 'Morphin' in analgetic or 'Morphine' in analgetic:
            dose = analgetic.split('Morphin')[-1].strip()
            df.at[i, 'morphine_dose'] += float(dose)

df['any_opiate_dose'] = df['morphine_dose'] + df['fentanyl_dose']
df['any_ketamine_dose'] = df['ketamine_dose'] + df['esketamine_dose']
df['any_opiate_given'] = (df['morphine_dose'] > 0) | (df['fentanyl_dose'] > 0)
df['any_ketamine_given'] = (df['ketamine_dose'] > 0) | (df['esketamine_dose'] > 0)
df['opiate_ketamine_combination'] = (df['any_opiate_given'] & df['any_ketamine_given']).astype(int)
df['no_analgesic'] = ((df['any_opiate_given'] == 0) & (df['any_ketamine_given'] == 0)).astype(int)

model_vars = [
    'vas_reduction', 'any_opiate_dose', 'any_ketamine_dose', 'age', 'NACA',
    'male_physician', 'male_patient', 'physician_age', 'physician_experience_years',
    'physician_anesthesiologist', 'physician_intensivist', 'physician_internist',
    'mission_duration', 'primary_mission', 'night_mission', 'winter_season',
    'winch_extraction', 'vas_scene', 'opiate_ketamine_combination', 'no_analgesic'
]
model_vars = [v for v in model_vars if v in df.columns]
df_clean = df[model_vars].dropna()
predictor_vars = [v for v in model_vars if v != 'vas_reduction']
predictor_vars = drop_zero_variance(df_clean, predictor_vars)

print(f'Medical patients included: {len(df_clean)}')
print(f'Mean VAS reduction: {df_clean["vas_reduction"].mean():.2f} ± {df_clean["vas_reduction"].std():.2f}')

In [ ]:
univariate_results = univariate_linear_regression(df_clean, 'vas_reduction', predictor_vars)
multivariate_results, multivariate_model = multivariate_linear_regression(df_clean, 'vas_reduction', predictor_vars)

univariate_results.sort_values('P_value')

In [ ]:
multivariate_results.sort_values('P_value')

In [ ]:
multivariate_results_normalized, _ = multivariate_linear_regression(
    df_clean, 'vas_reduction', predictor_vars, normalize=True
 )

summary_table = pd.DataFrame({
    'Variable': univariate_results['Variable'],
    'Univariate OR (95% CI)': univariate_results['Coef_CI'],
    'Univariate P-value': univariate_results['P_formatted'],
})

summary_table = summary_table.merge(
    multivariate_results[['Variable', 'Coef_CI', 'P_formatted']],
    on='Variable', how='left'
).rename(columns={
    'Coef_CI': 'Multivariate OR (95% CI)',
    'P_formatted': 'Multivariate P-value',
})

summary_table = summary_table.merge(
    multivariate_results_normalized[['Variable', 'Coef_CI', 'P_formatted']],
    on='Variable', how='left'
).rename(columns={
    'Coef_CI': 'Normalized OR (95% CI)',
    'P_formatted': 'Normalized P-value',
})

summary_table

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Forest plot using non-normalized data for medical population only
def create_medical_forest_plot():
    """
    Create a forest plot showing coefficients for medical population only
    using non-normalized predictors.
    """
    fig, ax = plt.subplots(figsize=(12, 10))

    # Variable labels for better presentation
    var_labels = {
        'any_opiate_dose': 'Total opioid dose (mg)',
        'any_ketamine_dose': 'Total ketamine dose (mg)',
        'age': 'Patient age (years)',
        'male_patient': 'Male patient',
        'male_physician': 'Male physician',
        'physician_age': 'Physician age (years)',
        'physician_experience_years': 'Physician experience (years)',
        'physician_anesthesiologist': 'Physician anesthesiologist',
        'physician_intensivist': 'Physician intensivist',
        'physician_internist': 'Physician internist',
        'mission_duration': 'Mission duration (minutes)',
        'NACA': 'NACA score',
        'primary_mission': 'Primary mission',
        'night_mission': 'Night mission',
        'winter_season': 'Winter season',
        'winch_extraction': 'Winch extraction',
        'vas_scene': 'NRS on scene',
        'opiate_ketamine_combination': 'Opioid + ketamine combination',
        'no_analgesic': 'No analgesic given',
    }

    if len(multivariate_results) > 0:
        # Prepare data from medical multivariate results
        variables = []
        coefs = []
        ci_lower = []
        ci_upper = []
        pvals = []

        # Sort by coefficient for better visualization
        medical_results_sorted = multivariate_results.sort_values('Coefficient', ascending=True)

        for _, row in medical_results_sorted.iterrows():
            var = row['Variable']
            variables.append(var_labels.get(var, var))
            coefs.append(row['Coefficient'])
            ci_lower.append(row['CI_lower'])
            ci_upper.append(row['CI_upper'])
            pvals.append(row['P_value'])

        y_positions = np.arange(len(variables))

        # Plot the forest plot
        for i, (coef_val, ci_l, ci_u, p_val) in enumerate(zip(coefs, ci_lower, ci_upper, pvals)):
            color = '#FF4444' if p_val < 0.05 else '#666666'
            size = 120 if p_val < 0.05 else 80

            # Plot point estimate
            ax.scatter(coef_val, y_positions[i], color=color, s=size, zorder=3, alpha=0.8)

            # Plot confidence interval
            ax.plot([ci_l, ci_u], [y_positions[i], y_positions[i]],
                   color=color, linewidth=2, alpha=0.7, zorder=2)

            # Plot CI caps
            cap_size = 0.1
            ax.plot([ci_l, ci_l], [y_positions[i]-cap_size, y_positions[i]+cap_size],
                   color=color, linewidth=2, alpha=0.7, zorder=2)
            ax.plot([ci_u, ci_u], [y_positions[i]-cap_size, y_positions[i]+cap_size],
                   color=color, linewidth=2, alpha=0.7, zorder=2)

            # Add coefficient and CI text
            ax.text(max(coefs) * 1.2 if max(coefs) != 0 else 0.5, y_positions[i],
                   f'{coef_val:.2f} ({ci_l:.2f} to {ci_u:.2f})',
                   va='center', fontsize=9, fontweight='bold' if p_val < 0.05 else 'normal')

        # Reference line at 0
        ax.axvline(x=0, color='black', linestyle='--', linewidth=1, alpha=0.8)

        # Formatting
        ax.set_xlabel('Coefficient (95% CI)', fontsize=14, fontweight='bold')
        ax.set_yticks(y_positions)
        ax.set_yticklabels(variables, fontsize=11)

        # Set x-axis limits with some padding
        x_min = min(min(ci_lower) * 1.1, min(coefs) * 1.1, -0.1)
        x_max = max(max(ci_upper) * 1.1, max(coefs) * 1.1, 0.1)
        ax.set_xlim(x_min, x_max)

        # Add grid
        ax.grid(True, alpha=0.3, axis='x')

        # remove upper and right spine
        ax.spines[['right', 'top']].set_visible(False)

        plt.tight_layout()
        plt.show()

        # Print summary
        print("\nForest Plot Summary - Medical Population:")
        print("=" * 50)
        print(f"Total variables analyzed: {len(variables)}")
        print(f"Significant associations (p < 0.05): {sum(1 for p in pvals if p < 0.05)}")
        print(f"Sample size: {len(df_clean)} medical patients")
        print(f"Mean VAS reduction: {df_clean['vas_reduction'].mean():.2f}")

        # List significant variables
        sig_vars = [(var, coef_val, p_val) for var, coef_val, p_val in zip(variables, coefs, pvals) if p_val < 0.05]
        if sig_vars:
            print("\nSignificant associations:")
            for var, coef_val, p_val in sig_vars:
                direction = "increase" if coef_val > 0 else "decrease"
                print(f"  • {var}: Coef = {coef_val:.2f} ({direction} in VAS reduction, p = {p_val:.3f})")
        else:
            print("\nNo variables reached statistical significance (p < 0.05)")
    else:
        print("No medical multivariate results available for forest plot")

    return fig

# Create the medical forest plot
fig = create_medical_forest_plot()

# Forest plot using normalized data for medical population only
def create_medical_forest_plot_normalized():
    """
    Create a forest plot showing coefficients for medical population only
    using normalized predictors (per 1 SD increase).
    """
    fig, ax = plt.subplots(figsize=(12, 10))

    # Variable labels for better presentation
    var_labels = {
        'any_opiate_dose': 'Total opioid dose (mg)',
        'any_ketamine_dose': 'Total ketamine dose (mg)',
        'age': 'Patient age (years)',
        'male_patient': 'Male patient',
        'male_physician': 'Male physician',
        'physician_age': 'Physician age (years)',
        'physician_experience_years': 'Physician experience (years)',
        'physician_anesthesiologist': 'Physician anesthesiologist',
        'physician_intensivist': 'Physician intensivist',
        'physician_internist': 'Physician internist',
        'mission_duration': 'Mission duration (minutes)',
        'NACA': 'NACA score',
        'primary_mission': 'Primary mission',
        'night_mission': 'Night mission',
        'winter_season': 'Winter season',
        'winch_extraction': 'Winch extraction',
        'vas_scene': 'NRS on scene',
        'opiate_ketamine_combination': 'Opioid + ketamine combination',
        'no_analgesic': 'No analgesic given',
    }

    if len(multivariate_results_normalized) > 0:
        # Prepare data from normalized medical multivariate results
        variables = []
        coefs = []
        ci_lower = []
        ci_upper = []
        pvals = []

        # Sort by coefficient for better visualization
        medical_results_sorted = multivariate_results_normalized.sort_values('Coefficient', ascending=True)

        for _, row in medical_results_sorted.iterrows():
            var = row['Variable']
            variables.append(var_labels.get(var, var))
            coefs.append(row['Coefficient'])
            ci_lower.append(row['CI_lower'])
            ci_upper.append(row['CI_upper'])
            pvals.append(row['P_value'])

        y_positions = np.arange(len(variables))

        # Plot the forest plot
        for i, (coef_val, ci_l, ci_u, p_val) in enumerate(zip(coefs, ci_lower, ci_upper, pvals)):
            color = '#FF4444' if p_val < 0.05 else '#666666'
            size = 120 if p_val < 0.05 else 80

            # Plot point estimate
            ax.scatter(coef_val, y_positions[i], color=color, s=size, zorder=3, alpha=0.8)

            # Plot confidence interval
            ax.plot([ci_l, ci_u], [y_positions[i], y_positions[i]],
                   color=color, linewidth=2, alpha=0.7, zorder=2)

            # Plot CI caps
            cap_size = 0.1
            ax.plot([ci_l, ci_l], [y_positions[i]-cap_size, y_positions[i]+cap_size],
                   color=color, linewidth=2, alpha=0.7, zorder=2)
            ax.plot([ci_u, ci_u], [y_positions[i]-cap_size, y_positions[i]+cap_size],
                   color=color, linewidth=2, alpha=0.7, zorder=2)

            # Add coefficient and CI text
            ax.text(max(coefs) * 1.2 if max(coefs) != 0 else 0.5, y_positions[i],
                   f'{coef_val:.2f} ({ci_l:.2f} to {ci_u:.2f})',
                   va='center', fontsize=9, fontweight='bold' if p_val < 0.05 else 'normal')

        # Reference line at 0
        ax.axvline(x=0, color='black', linestyle='--', linewidth=1, alpha=0.8)

        # Formatting
        ax.set_xlabel('Coefficient per 1 SD increase (95% CI)', fontsize=14, fontweight='bold')
        ax.set_yticks(y_positions)
        ax.set_yticklabels(variables, fontsize=11)

        # Set x-axis limits with some padding
        x_min = min(min(ci_lower) * 1.1, min(coefs) * 1.1, -0.1)
        x_max = max(max(ci_upper) * 1.1, max(coefs) * 1.1, 0.1)
        ax.set_xlim(x_min, x_max)

        # Add grid
        ax.grid(True, alpha=0.3, axis='x')

        # remove upper and right spine
        ax.spines[['right', 'top']].set_visible(False)

        plt.tight_layout()
        plt.show()

        # Print summary
        print("\nForest Plot Summary - Medical Population (Normalized):")
        print("=" * 50)
        print(f"Total variables analyzed: {len(variables)}")
        print(f"Significant associations (p < 0.05): {sum(1 for p in pvals if p < 0.05)}")
        print(f"Sample size: {len(df_clean)} medical patients")
        print(f"Mean VAS reduction: {df_clean['vas_reduction'].mean():.2f}")

        # List significant variables
        sig_vars = [(var, coef_val, p_val) for var, coef_val, p_val in zip(variables, coefs, pvals) if p_val < 0.05]
        if sig_vars:
            print("\nSignificant associations:")
            for var, coef_val, p_val in sig_vars:
                direction = "increase" if coef_val > 0 else "decrease"
                print(f"  • {var}: Coef = {coef_val:.2f} ({direction} in VAS reduction, p = {p_val:.3f})")
        else:
            print("\nNo variables reached statistical significance (p < 0.05)")
    else:
        print("No medical normalized multivariate results available for forest plot")

    return fig

# Create the medical normalized forest plot
norm_fig = create_medical_forest_plot_normalized()

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt

output_dir = Path("/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/analgesia/medical_analgesia")
output_dir.mkdir(parents=True, exist_ok=True)
notebook_tag = "table4_reduction_in_VAS"

# Save summary table
if 'summary_table' in globals():
    try:
        summary_table.to_excel(output_dir / f"{notebook_tag}_summary_table.xlsx", index=False)
    except Exception as exc:
        print(f"Could not save Excel summary table: {exc}")

# Save named figures if present
saved_fig_nums = set()
if 'fig' in globals():
    saved_fig_nums.add(fig.number)
    fig.savefig(output_dir / f"{notebook_tag}_forest_plot.png", dpi=300, bbox_inches="tight")
if 'norm_fig' in globals():
    saved_fig_nums.add(norm_fig.number)
    norm_fig.savefig(output_dir / f"{notebook_tag}_forest_plot_normalized.png", dpi=300, bbox_inches="tight")

# Save any other open matplotlib figures
for idx, num in enumerate(plt.get_fignums(), start=1):
    if num in saved_fig_nums:
        continue
    plt.figure(num).savefig(output_dir / f"{notebook_tag}_figure_{idx}.png", dpi=300, bbox_inches="tight")